In [212]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import tensorflow as tf
import pathlib
import PIL
import time
import zipfile
import random
from tensorflow import keras as ks
from tensorflow.keras.layers import *
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from fastai.vision.all import *

In [213]:
train_df = pd.read_csv("../input/makerere-passion-fruit-disease-detection-challenge/Train (11).csv")
test_df = pd.read_csv("../input/makerere-passion-fruit-disease-detection-challenge/Test (12).csv")
sub=pd.read_csv("../input/makerere-passion-fruit-disease-detection-challenge/Sample_submission (5).csv")

In [214]:
train=[]
test=[]
diir="../input/makerere-passion-fruit-disease-detection-challenge/Train_Images/Train_Images/"
for i in os.listdir(diir):
    path=diir+i
    train.append(path)
dirr="../input/makerere-passion-fruit-disease-detection-challenge/Test_Images (1)/Test_Images/"
for i in os.listdir(dirr):
    path=dirr+i
    test.append(path)

In [215]:
len(test)

931

In [216]:
train_df.head()

,Image_ID,class,xmin,ymin,width,height
0,ID_007FAIEI,fruit_woodiness,87.0,87.5,228.0,311.0
1,ID_00G8K1V3,fruit_brownspot,97.5,17.5,245.0,354.5
2,ID_00WROUT9,fruit_brownspot,156.5,209.5,248.0,302.5
3,ID_00ZJEEK3,fruit_healthy,125.0,193.0,254.5,217.0
4,ID_018UIENR,fruit_brownspot,79.5,232.5,233.5,182.0


In [217]:
train_df.columns

Index(['Image_ID', 'class', 'xmin', 'ymin', 'width', 'height'], dtype='object')

In [218]:
clas = train_df["class"][:3001]
xmin = train_df["xmin"][:3001]
ymin = train_df["ymin"][:3001]
width = train_df["width"][:3001]
height = train_df["height"][:3001]

In [219]:
# df=pd.DataFrame({'path':train, 'class':clas, 'xmin':xmin, 'ymin':ymin, 'width':width, 'height':height})

In [242]:
df=pd.DataFrame({'path':train, 'class':clas})

SyntaxError: invalid syntax (684304141.py, line 1)

In [243]:
df['fold'] = pd.Series(dtype='int')

In [ ]:
test_d=pd.DataFrame({'path':test})

In [ ]:
df

In [ ]:
from PIL import Image
img = Image.open(df.loc[0,"path"])
img

In [ ]:
train_datagenerator = ImageDataGenerator(
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    rescale=1./255, 
    rotation_range=40, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=.2, 
    zoom_range=0.2,
    horizontal_flip=True, 
    fill_mode='nearest',
    validation_split=0.1)

In [221]:
df.columns

Index(['path', 'class'], dtype='object')

In [222]:
# train_generator = train_datagenerator.flow_from_dataframe(
#     df, 
#     x_col='path', 
#     y_col=['class', 'xmin', 'ymin', 'width', 'height'],
#     target_size=(330, 330), 
#     color_mode='rgb', 
#     class_mode='raw', 
#     batch_size=32, 
#     shuffle=True, 
#     seed=1,
#     subset='training')
# validation_generator = train_datagenerator.flow_from_dataframe(
#     df , 
#     x_col='path',
#     y_col=['class', 'xmin', 'ymin', 'width', 'height'],
#     target_size=(330, 330),
#     color_mode='rgb', 
#     class_mode='raw', 
#     batch_size=4, 
#     shuffle=True,
#     seed=1, 
#     subset='validation'
# )

In [223]:
%%capture
!pip install iterative-stratification
from sklearn import model_selection
from tqdm.auto import tqdm
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [224]:
df.columns

Index(['path', 'class'], dtype='object')

In [225]:
# def create_folds(data, num_splits):
#     data["fold"] = -1

#     mskf = MultilabelStratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)
#     labels = ['class']
#     data_labels = data[labels].values

#     for f, (t_, v_) in enumerate(mskf.split(data, data_labels)):
#         data.loc[v_, "fold"] = f

#     return data
# df = create_folds(df, num_splits=6)
# print("Folds created successfully")

In [226]:
# classes_la = {"fruit_brownspot": 1, "fruit_healthy": 2, "fruit_woodiness":3}

# df["class"] = df["class"].apply(lambda x: classes_la[x])

In [227]:
df

,path,class
0,../input/makerere-passion-fruit-disease-detection-challenge/Train_Images/Train_Images/ID_6Y096CZN.jpg,fruit_woodiness
1,../input/makerere-passion-fruit-disease-detection-challenge/Train_Images/Train_Images/ID_PMA5U1KL.jpg,fruit_brownspot
2,../input/makerere-passion-fruit-disease-detection-challenge/Train_Images/Train_Images/ID_05R05ZLJ.jpg,fruit_brownspot
3,../input/makerere-passion-fruit-disease-detection-challenge/Train_Images/Train_Images/ID_KSKXJCK0.jpg,fruit_healthy
4,../input/makerere-passion-fruit-disease-detection-challenge/Train_Images/Train_Images/ID_K88UG3YE.jpg,fruit_brownspot
...,...,...
2996,../input/makerere-passion-fruit-disease-detection-challenge/Train_Images/Train_Images/ID_IPXI35JJ.jpg,fruit_woodiness
2997,../input/makerere-passion-fruit-disease-detection-challenge/Train_Images/Train_Images/ID_X1MYEGIE.jpg,fruit_brownspot
2998,../input/makerere-passion-fruit-disease-detection-challenge/Train_Images/Train_Images/ID_DPV8ZOVG.jpg,fruit_brownspot
2999,../input/makerere-passion-fruit-disease-detection-challenge/Train_Images/Train_Images/ID_CM1DPVY0.jpg,fruit_woodiness


****

In [228]:
from fastai.vision.all import *
import warnings 
import albumentations as A
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold

In [229]:
IMG_HEIGHT = IMG_WIDTH = 224
IMAGENET_MEAN = (0.485, 0.456)
IMAGENET_STD = (0.229, 0.224)
INCEPTION_MEAN = INCEPTION_STD = (0.5, 0.5)


class AlbumentationsTransform(RandTransform):
    "A transform handler for multiple `Albumentation` transforms"
    split_idx, order = None, 3
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        
        return aug_img


def get_train_transforms(mean_std):
    augmentations = [
        A.HorizontalFlip(p=.5),
        A.VerticalFlip(p=.5),
        A.RandomRotate90(p=.5),
        A.ImageCompression(quality_lower=99, quality_upper=100),
        A.ShiftScaleRotate(
            shift_limit=0.2, scale_limit=0.2, 
            rotate_limit=45, border_mode=0, p=.5
        ),
        A.Resize(IMG_HEIGHT, IMG_WIDTH),
        A.Cutout(
            max_h_size=int(IMG_HEIGHT),
            max_w_size=int(IMG_WIDTH),
            num_holes=1,
            p=.75,
        ),
    ]
    if mean_std=='imagenet':
        augmentations.append(A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD))
    elif mean_std=='inception':
        augmentations.append(A.Normalize(mean=INCEPTION_MEAN, std=INCEPTION_STD))
    else:
        augmentations.append(A.Normalize(mean=0, std=1))

    augmentations.append(ToTensorV2())
    return A.Compose(augmentations)


def get_valid_transforms(mean_std):
    augmentations = [A.Resize(IMG_HEIGHT, IMG_WIDTH)]
    if mean_std=='imagenet':
        augmentations.append(A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD))
    elif mean_std=='inception':
        augmentations.append(A.Normalize(mean=INCEPTION_MEAN, std=INCEPTION_STD))
    else:
        augmentations.append(A.Normalize(mean=0, std=1))

    augmentations.append(ToTensorV2())
    return A.Compose(augmentations)

def get_item_tfms(mean_std='imagenet'):
    return [AlbumentationsTransform(get_train_transforms(mean_std), get_valid_transforms(mean_std))]

In [230]:
class Head(nn.Module):
    def __init__(self, in_features, out_features):
        super(Head, self).__init__()
        self.head = nn.Linear(in_features=in_features, out_features=out_features)

    def forward(self, x):
        return self.head(x)


class Net(nn.Module):
    def __init__(
        self,
        base_model="resnet50",
        pretrained=True,
        checkpoint_path=None,
        num_classes=3,
    ):
        super(Net, self).__init__()

        self.backbone = timm.create_model(
            base_model, pretrained=pretrained, checkpoint_path=checkpoint_path
        )
        in_features = self.backbone.get_classifier().in_features
        self.backbone.reset_classifier(num_classes=0, global_pool="avg")
        self.neck = Head(in_features=in_features, out_features=in_features)
        self.head = Head(in_features=in_features, out_features=num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.neck(x)
        x = self.head(x)
        x = F.log_softmax(x, dim=1)
        return x

In [231]:
# classes=['class', 'xmin', 'ymin', 'width', 'height']

In [232]:
# def get_y(df):
#     classes=['class', 'xmin', 'ymin', 'width', 'height']
#     return df[classes].values.astype(np.int32) 

In [233]:
# get_y

In [234]:
# def get_dls(mean_std='imagenet'):
#     clas_block = DataBlock(blocks=(ImageBlock,MultiCategoryBlock(encoded=True,vocab=classes)),
#                         get_x=ColReader("path"),
#                         get_y=ColReader("class"),
#                         item_tfms=get_item_tfms(mean_std),
#                         )
#     dls = clas_block.dataloaders(df, bs=16)
#     dls.rng.seed(21)

#     return dls

In [245]:
import gc
import inspect
from fastai.metrics import accuracy
from fastai.optimizer import OptimWrapper
from torch import optim
from functools import partial
from transformers import AdamW


def free_memory(to_delete: list):
    calling_namespace = inspect.currentframe().f_back

    for _var in to_delete:
        calling_namespace.f_locals.pop(_var, None)
        gc.collect()
        torch.cuda.empty_cache()

def get_dls(fold=0, mean_std='imagenet'):
    clas_block = DataBlock(blocks=(ImageBlock, CategoryBlock),
                        splitter=MaskSplitter(df.fold == fold),
                        get_x=ColReader("path"),
                        get_y=ColReader("class"),
                        item_tfms=get_item_tfms(mean_std),
                        )
    dls = clas_block.dataloaders(df, bs=32)
    dls.rng.seed(21)

set_seed(21, reproducible=True)

In [246]:
# df=df.drop("fold", axis=1)

In [247]:
!pip install timm

In [248]:
import timm
predictions = []
for fold in [1]:
    print('*'*25+f"Fold {fold}"+'*'*25)
    m_name = f'vfold-{fold}'
    dls = get_dls(fold ,'imagenet')
    learn = Learner(dls, Net(base_model='vit_tiny_patch16_224'), 
                    loss_func=CrossEntropyLossFlat(), metrics=accuracy,
                    opt_func=partial(OptimWrapper, opt=optim.Adam), 
                    cbs=[SaveModelCallback(reset_on_fit=False, fname=m_name)]
            )
    
    learn.fit_one_cycle(20, 2e-4)
    learn.save(m_name+'_stage-1')
    learn = learn.load(m_name)
    learn.freeze_to(-5)
    learn.fit_one_cycle(4, 2e-6)
    learn.save(m_name+'_stage-2');

    model = learn.load(m_name)
    preds, _ = model.tta(dl=dls.test_dl(test_df), n=5)
    predictions.append(preds)

    free_memory([model, learn, preds, dls])

*************************Fold 1*************************
Could not do one pass in your dataloader, there is something wrong in it. Please see the stack trace below:


ValueError: operands could not be broadcast together with shapes (224,224,3) (2,) (224,224,3) 